In [74]:
import json
from bs4 import BeautifulSoup
from collections import OrderedDict


# set up the corpus object
# and wtf does it being a translation suggest we do :O
corpus = OrderedDict()
corpus["author"] = [{"name": "Ovid"}, {"slug": "ovid"}, {"full name": "Publius Ovidius Naso"}, {"abbreviation": "Ov."}]
# add the empty works list, which is what we'll populate
works_list = []


# includes Amores, Ars Amatoria, and Remedia
html_file = "Kline_Love_Poems.html"

html_doc = open(html_file).read()
soup = BeautifulSoup(html_doc, 'html.parser')

# works = []
# article tags surround works
# all the rest of the tags are unnested :(
for index, work in enumerate(soup.findAll('article')):
    # ignore the preface material
    if index==0:
        pass
    else:
        # the h1 is the title
        print work.find('h1').text
        # only h2s with this class indicate Book titles
        for index, book in enumerate(work.findAll('h2', class_='text-align-center')):
            print index
            print book.text
            for index, poem in enumerate(book.find_next_siblings('h3')):
                print index
                print poem.text
                end_of_poem = False
                for index, line in enumerate(poem.find_next_siblings()):
                    #  print first line
                    if index == 0:
                        print line.text
                    line = line.find_next_sibling()
                    try:
                        tag_name = line.name
                    except AttributeError:
                        tag_name = ""
                    if tag_name == "p":
                        print line.text
                    else:
                        print "*****"
                        break

# structure notes
#poems have individual anchor links , so for Amores, start anchor_Toc520535255 end anchor_Toc520536672
# basic html structure -- not much nesting :/
# <article> #indicates work
# <h1 id="Amoreshome">The Amores</h1> #indicates title of work
# <h2 class="text-align-center" id="Amoresbk1">Book I</h2> #indicates book num, has class, has hr beforehand
# <h2>Contents</h2> #hanging out, does not have class
# <h3><a id="anchor_Toc520535255">His Epigram</a></h3> #actual poems have h3 at head, no surrounding tags
# <h2 class="text-align-center" id="Amoresbk2">Book II</h2>
# <h2>Contents</h2>
# <h3><a id="anchor_Toc520535255">His Epigram</a></h3>
# <h2 class="text-align-center" id="Amoresbk3">Book III</h2>
# <h2>Contents</h2>
# </article> #ends just after p of last poem
# h2 Amatoriabk1
# end of book:
# <hr />
# <h2 class="text-align-center" id="Amoresbk2">Book II</h2>


# other notes
# https://stackoverflow.com/questions/11647348/find-next-siblings-until-a-certain-one-using-beautifulsoup

The Amores
0
Book I
0
Elegy 0:His Epigram
We who were once five books are now three:
The author preferred the work this way.
Now, if it’s no joy to you to read us,
still it’s a lighter punishment with two books less.
 
*****
1
Book I Elegy I : The Theme of Love
Just now, I was preparing to start with heavy fighting
and violent war, with a measure to fit the matter.
Good enough for lesser verse – laughed Cupid
so they say, and stole a foot away.
‘Cruel boy, who gave you power over this song?
Poets are the Muses’, we’re not in your crowd.
What if Venus snatched golden Minerva’s weapons,
while golden Minerva fanned the flaming fires?
Who’d approve of Ceres ruling the wooded hills,
with the Virgin’s quiver to cultivate the fields?
Who’d grant long-haired Phoebus a sharp spear,
while Mars played the Aonian lyre?
You’ve a mighty kingdom, boy, and too much power,
ambitious one, why aspire to fresh works?
Or is everything yours? Are Helicon’s metres yours?
Is even Phoebus’s lyre now barely his

Away with these surly tablets of funereal wood,
and you, wax, filled with your negative message! –
Extracted I bet from honey of long hemlock flowers
made by the infamous Corsican bees.
Just as if you’d blushed, steeped in deep dye –
that colour indeed was truly bloody.
Useless wood, I’ll throw you out at the crossroads,
so the weight of a passing wheel can smash you!
Even the man who carved you for use, from the tree,
I’m convinced the man had impure hands.
That tree held some wretch hung by the neck,
it offered itself as dread executioner’s crosses:
it gave vile shade to the screeching owls,
and carried their eggs and vultures in its branches.
Madman, did I give these to my lady, trusting
my love to them, to carry my gentle words?
This wax is more fitted to garrulous words of bail,
to be read aloud by some hard mouthed attorney:
or better to throw these tablets among the accounts,
where a miser goes weeping for his lost wealth.
So I judge you, two-faced things by nature.
The number i

I’d not fear the monsters yelping from Scylla’s virgin groin,
nor would I fear your folds, curved Cape Malea:
nor Charybdis’s mouth glutted with wrecked ships
spewing out and sucking back the flooding waters.
But should Neptune’s stormy powers triumph,
and the gods that aid us be carried off by the waves,
you’d throw your white arms about my shoulders:
I’d bear your sweet body’s burden easily.
Young Leander often swam the waves seeking Hero,
then swam again, but the sea-road was dark.
But without you here, though the busy vineyards
occupy me, though the countryside’s flowing with rivers,
and countrymen summon flowing water to their streams,
and cool breezes caress the leafy trees,
I don’t think of celebrating Sulmo’s healthiness,
that’s its my native place, ancestral country –
it’s Scythia, with wild Cilician pirates, painted Britons,
or the Promethean rocks dyed red with blood.
Elm loves vines, vines never desert their elm:
why should I be so often parted from my girl?
And you swore t

Book II Elegy III: The Eunuch
Ah me, that you, neither man nor woman, serve the lady
you who can’t know the mutual delights of Venus!
Whoever first cut off a boy’s genitals, that one,
who made the wound, should suffer it himself.
You’d be more gently compliant, facilitate my requests,
if you’d ever glowed with love before.
You weren’t born to ride a horse, or use heavy weapons:
a warlike spear would not be fitting in your hand.
Let men handle that: you can forget manly hopes.
your camp is with your lady.
Work your service there, you’ll benefit from her thanks:
What use would you be if you didn’t have her?
She’s lovely, the right age for play:
a disgrace to waste that beauty through sheer neglect.
She could have deceived you, however irksome you are:
Two, who want to, won’t fail to achieve it.
Still as it was fitting to try a request, so I’m asking,
while you’ve a good chance of gaining a reward.
 
*****
3
Book II Elegy IV: His Susceptibility
I wouldn’t dare defend my suspect morals
or 

the word threshing-floor was unknown on the Earth,
but oak-trees, the first oracles, carried acorns:
these and tender herbs in the grass were our food.
Ceres first taught the seeds to swell in the fields,
and first with sickles cut the ripened sheaves:
first bowed the necks of oxen under the yoke,
and scarred the ancient earth with curved blade.
Can anyone believe she delights in lovers’ tears
that right worship lies in torment and lonely beds?
Still, though she loves fertile fields, she’s no rustic,
nor does she have a heart bereft of love.
The Cretans are witness – Cretans’ don’t always lie.
Crete was proud to nurse the infant Jove.
There, he who steers the world’s starry courses,
sucked milk, with tender mouth as a little child.
Proof from a mighty witness: witnessed by his praise.
I think Ceres might confess to the charge I make.
She saw Iasus on the slopes of Cretan Mount Ida,
slaughtering the game with unerring hand.
She saw him, and flames pierced her to the marrow,
from there, 

Let me spread sail and enjoy the flowing breezes,
or, if I may not, to want what I’m forced to love.
 
*****
12
Book III Elegy XII: It Serves Me Right!
What day was it, dark bird, when you sounded
your omen for this eternally melancholy lover?
What star should I believe has opposed my destiny,
what god should I complain of, warring against me?
She who was once spoken of as mine, whom I loved,
first, alone, I fear, along with many others, I consider mine.
Am I mistaken, or have my books made her famous?
so it shall be – she’ll be advertised by my art.
And it serves me right! For didn’t I trumpet her beauty?
It’s my fault if the girl’s been rendered marketable.
It pleased me to be go-between, guide to lovers I attracted,
the entrance was thrown open by my hand.
And I doubt the use of verse that’s always harmed me:
it made men envious of my success.
Despite Thebes, and Troy, and Caesar’s actions,
only Corinna inspired my genius.
I wish a hostile Muse had struck my verse,
that Apollo had f

as if struck, as they say, by the horns of the Boeotian god.
Wronged by her husband, her marriage violated,
savage Medea avenged herself through her children.
Another fatal mother was that swallow, you see there:
look, her breast carries the stain of blood.
Well-founded and firm loves have been dissolved so:
these are crimes to make cautious men afraid.
Not that my censure condemns you to only one girl:
the gods forbid! A wife could hardly expect that.
Indulge, but secretly veil your sins, with restraint:
it’s no glory to you to be seeking out wrongdoing.
Don’t give gifts another girl could spot,
or have set times for your assignations.
And lest a girl catch you out in your favourite haunts
don’t meet all of them in one place.
And always look closely at your wax tablets, whenever you write:
lest much more is read there than you sent.
Wounded, Venus takes up just arms, and hurls her dart,
and makes you lament, as she is lamenting.
While Agamemnon was satisfied with one woman, Clytemnest

she’s worthy of sleeping with anyone who’ll have her.
And it’s not safe to fall asleep at table:
many shameful things usually happen in sleep.
 
*****
56
Book III Part XVIII: And So To Bed
To have been taught more is shameful: but kindly Venus
said: ‘What’s shameful is my particular concern.’
Let each girl know herself: adopt a reliable posture
for her body: one layout’s not suitable for all.
She who’s known for her face, lie there face upwards:
let her back be seen, she who’s back delights.
Milanion bore Atalanta’s legs on his shoulders:
if they’re good looking, that mode’s acceptable.
Let the small be carried by a horse: Andromache,
his Theban bride, was too tall to straddle Hector’s horse.
Let a woman noted for her length of body, 
press the bed with her knees, arch her neck slightly.
She who has youthful thighs, and faultless breasts,
the man might stand, she spread, with her body downwards.
Don’t think it shameful to loosen your hair, like a Maenad,
and throw back your head with i

snatch the swift fruits, that fly, in your hand.
 
*****
33
Book III Part XIV: Use Jealousy and Fear
Let all be betrayed: I’ve unbarred the gates to the enemy:
and let my loyalty be to treacherous betrayal.
What’s easily given nourishes love poorly:
mingle the odd rejection with welcome fun.
Let him lie before the door, crying: ‘Cruel entrance!,
pleading very humbly, threatening a lot too.
We can’t stand sweetness: bitterness renews our taste:
often a yacht sinks swamped by a favourable wind:
this is what bitter wives can’t endure:
their husbands can come to them when they wish:
add a closed door and a hard-mouthed janitor,
saying: ‘You can’t,’ and love will touch you too.
Drop the blunted foils now: fight with blades:
no doubt I’ll be attacked with my own weapons.
Also when the lover you’ve just caught falls into the net,
let him think that only he has access to your room.
Later let him sense a rival, the bed’s shared pact:
remove these arts, and love grows old.
The horse runs swiftly